<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/Dynamics_Discretized_PFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

In [227]:
q=100.
n=50
V=100/n
rho=1000
C=0.239
H=5e4
ER = 8750
k0 = 7.2e10
UA = 5e4/n  # J/(min K m3)

In [228]:
def rhs(t,vec, Ain, Tin, Tc):
    A,T = vec.reshape(2,-1)
    k = k0 * np.exp(-8750/T)
    dA0 = q/V*(Ain - A[0]) - k[0]*A[0]
    dT0 = q/V*(Tin - T[0]) + H*k[0]*A[0]/(rho*C) - UA*(T[0]- Tc)/(rho*V*C)

    dA = q/V*(A[:-1]-A[1:]) - k[1:]*A[1:]
    dT = q/V*(T[:-1] - T[1:]) + H*k[1:]*A[1:]/(rho*C) - UA*(T[1:]- Tc)/(rho*V*C)
    return np.r_[dA0, dA, dT0, dT]

In [229]:
A_init = np.tile(0.5, n)
T_init = np.tile(350., n)
vec_init = np.r_[A_init, T_init]

In [247]:
count = np.arange(n)+1
tend=2
sol=solve_ivp(rhs, (0,tend), vec_init, method='Radau', dense_output=True, args=(0.5, 350., 300.))
tplot=np.linspace(0,tend,101)
A,T=sol.sol(tplot).reshape(2,-1,101)
fig=make_subplots(rows=2,cols=2)
for i in range(0,n,5):
    fig.add_scatter(x=tplot, y=A[i], mode='lines', row=1,col=1, name=f'A{i+1}')
    fig.add_scatter(x=tplot, y=T[i], mode='lines', row=1,col=2, name=f'T{i+1}')

for i in range(0,T.shape[1],5):
    fig.add_scatter(x=count, y=A[:,i],row=2,col=1, mode='lines', name=f'A{tplot[i]}')
    fig.add_scatter(x=count, y=T[:,i],row=2,col=2, mode='lines', name=f'T{tplot[i]}')
fig.update_layout(width=800,height=800,template='plotly_dark', showlegend=False)
fig.show()
Asteady = A[:,-1]
Tsteady = T[:,-1]

In [248]:
tend=2
sol=solve_ivp(rhs, (0,tend), np.r_[Asteady,Tsteady], method='Radau', dense_output=True, args=(0.5, 355., 300.))
tplot=np.linspace(0,tend,101)
A,T=sol.sol(tplot).reshape(2,-1,101)
fig=make_subplots(rows=2,cols=2)
for i in range(0,n,5):
    fig.add_scatter(x=tplot, y=A[i], mode='lines', row=1,col=1, name=f'A{i+1}')
    fig.add_scatter(x=tplot, y=T[i], mode='lines', row=1,col=2, name=f'T{i+1}')

for i in range(0,T.shape[1],5):
    fig.add_scatter(x=count, y=A[:,i],row=2,col=1, mode='lines', name=f'A{tplot[i]}')
    fig.add_scatter(x=count, y=T[:,i],row=2,col=2, mode='lines', name=f'T{tplot[i]}')
fig.update_layout(width=800,height=800,template='plotly_dark', showlegend=False)